# Lab 1: Deploy Azure AI Foundry Resources

## Overview

In this notebook, we will deploy Azure resources to build a multi-agent system based on Azure AI Foundry.

### Resources to Deploy

- Azure AI Foundry Project
- Azure AI Search (for RAG knowledge base)
- Azure Container Apps Environment (for agent hosting)
- Azure Container Registry (for image storage)
- Azure Key Vault (for secret management)
- Azure Storage Account (for data storage)
- Azure OpenAI Service (for GPT-4 model)

### Architecture

```
┌────────────────────────────────────────────────────────────┐
│                 Multi-Agent System                         │
│                                                            │
│  ┌─────────────────────────────────────────────┐          │
│  │          Main Agent                         │          │
│  │  (Task Analysis & Routing)                  │          │
│  └────────────┬────────────────┬────────────────┘          │
│               │                │                           │
│       ┌───────▼──────┐  ┌──────▼──────────┐               │
│       │  Tool Agent  │  │  Research       │               │
│       │  (MCP)       │  │  Agent (RAG)    │               │
│       └──────┬───────┘  └────────┬────────┘               │
│              │                   │                         │
│       ┌──────▼───────┐    ┌──────▼─────────┐              │
│       │  MCP Server  │    │  Azure AI      │              │
│       │  (Weather)   │    │  Search (RAG)  │              │
│       └──────────────┘    └────────────────┘              │
└────────────────────────────────────────────────────────────┘
```

### Learning Objectives

1. ✅ Deploy infrastructure using Azure Developer CLI (azd)
2. ✅ Understand Infrastructure as Code based on Bicep templates
3. ✅ Understand Azure AI Foundry resource structure
4. ✅ Verify deployed resources and collect connection information

---

## ⚙️ Before You Start

**Select Python kernel:**

1. Click **"Select Kernel"** in the top right corner of the notebook
2. Select **"Python Environments..."**
3. Select **`.venv (Python 3.x.x)`** (virtual environment created in project root)

> 💡 **GitHub Codespaces**: In Codespaces, the `.venv` environment is created automatically.  
> If you don't see `.venv`, create it in the terminal with `python -m venv .venv`.

---

## 1. Prerequisites Check

Check if the following tools are installed:

- Python 3.9 or later
- Azure CLI
- Azure Developer CLI (azd)
- Docker (required for Container Apps deployment)

In [ ]:
import sys, subprocess, os
import platform

# Set PATH based on OS (supports macOS, Linux, Codespaces)
system = platform.system()
if system == 'Darwin':  # macOS
    # Add Homebrew paths (Intel & Apple Silicon)
    extra_paths = '/opt/homebrew/bin:/usr/local/bin'
elif system == 'Linux':  # Linux / Codespaces
    # Common Linux binary paths
    extra_paths = '/usr/local/bin:/usr/bin:/home/codespace/.local/bin'
else:  # Windows
    extra_paths = ''

if extra_paths:
    os.environ['PATH'] = extra_paths + ':' + os.environ.get('PATH', '')

def check(cmd, name):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, timeout=3, env=os.environ)
        print(f"{'✓' if result.returncode == 0 else '✗'} {name}")
    except Exception as e:
        print(f"✗ {name}")

print("=== Prerequisites Check ===")
print(f"✓ Python {sys.version.split()[0]} ({system})")
check("az --version", "Azure CLI")
check("azd version", "Azure Developer CLI")
check("docker --version", "Docker")
print("="*50)

## 2. Azure Authentication

Log in to Azure and set up your subscription. A browser will open to proceed with authentication.

---

### 🚨 Required Reading for GitHub Codespaces Users

In the Codespaces environment, **the browser will not open automatically**.  
If authentication fails when running the cell below, run the following commands directly in the **terminal**:

```bash
az login --use-device-code
azd auth login --use-device-code
```

These commands generate a **device code** that allows you to authenticate in another browser.

⚠️ **Important**: If you run the above commands in the terminal, **skip the next code cell (Azure Authentication)** and start from the cell after that (Verify Authentication).

---

### Tenant ID Configuration Guide

**In most cases**: You don't need to specify a tenant ID. Leave the `tenant_id` variable as `"<YOUR_TENANT_ID>"` or `None` and run the cell.

**When you need a tenant ID**:
- ✅ When you have access to multiple organization (company) Azure tenants
- ✅ When you need to work only with a specific organization's resources
- ✅ When you encounter "multiple tenants" related errors during login

**How to find your tenant ID**:
- Azure Portal → Azure Active Directory → Overview → Copy Tenant ID
- Or contact your organization administrator

In [ ]:
import subprocess, json

print("=== Azure Authentication ===\n🔐 Browser will open for authentication...\n")

# Enter your tenant ID here (optional)
# Example: tenant_id = "16b3c013-d300-468d-ac64-7eda0820b6d3"
tenant_id = "<YOUR_TENANT_ID>"  # Or set to None to use default tenant

# Log in to the specified tenant if tenant ID is set
if tenant_id and tenant_id != "<YOUR_TENANT_ID>":
    az = subprocess.run(f"az login --tenant {tenant_id}", shell=True, capture_output=True, text=True)
else:
    az = subprocess.run("az login", shell=True, capture_output=True, text=True)

print(f"{'✅' if az.returncode == 0 else '❌'} Azure CLI")

azd = subprocess.run("azd auth login", shell=True, capture_output=True, text=True)
print(f"{'✅' if azd.returncode == 0 else '❌'} Azure Developer CLI")

print("="*50)

In [ ]:
import subprocess, json

print("=== Verifying Azure Authentication ===\n")

result = subprocess.run("az account show", shell=True, capture_output=True, text=True, timeout=5)

if result.returncode == 0:
    account = json.loads(result.stdout)
    print(f"✅ Authenticated as: {account['user']['name']}")
    print(f"   Subscription: {account['name']}")
else:
    print("❌ Not authenticated - Run previous cell")

print("="*50)

## 3. Environment Configuration

Set up the environment for Azure Developer CLI.

In [ ]:
# Move to project root directory (use absolute path)
import os
from pathlib import Path

# Project root directory (notebook is located in root)
project_root = Path.cwd()

os.chdir(project_root)
print(f"✓ Project root: {os.getcwd()}")

# Verify directory structure
checks = [
    ('infra/main.bicep', 'Bicep templates'),
    ('azure.yaml', 'azure.yaml'),
    ('src/foundry_agent', 'Agent source code')
]

for path, name in checks:
    if os.path.exists(path):
        print(f"✓ {name} found")
    else:
        print(f"✗ {name} NOT found")
        
print("="*50)

> **🌍 Important: Change region in just 1 place here!**  
> Just change the `location` variable in the cell below.  
> Only modify when you have quota limitations or region constraints.  
> 
> **Recommended regions:** `eastus` (default) → Use `eastus2`, `westus`, `swedencentral` if quota is insufficient

In [ ]:
import random, string, json

# Get current Azure subscription information
subscription_result = subprocess.run("az account show", shell=True, capture_output=True, text=True)

if subscription_result.returncode != 0:
    print("❌ Not authenticated! Please run Cell 5 (Azure Authentication) first.")
    raise Exception("Azure authentication required")

account_info = json.loads(subscription_result.stdout)
subscription_id = account_info['id']
subscription_name = account_info['name']

print(f"📋 Using Subscription: {subscription_name}")
print(f"   ID: {subscription_id}\n")

# =============================================================================
# 🌍 Region Configuration - Change this 1 place only if quota is insufficient!
# =============================================================================

location = "eastus"  # 👈 Change only if quota is insufficient! (e.g., 'eastus2', 'westus', 'swedencentral')

# Generate environment name (automatic)
random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))
environment_name = f"aiagent-{random_suffix}"

print(f"🌍 Region Configuration")
print("=" * 50)
print(f"Environment Name: {environment_name}")
print(f"Location:         {location}")
print("=" * 50)
print("\n💡 Tip: If you encounter quota errors:")
print("   1. Change the location variable above to 'eastus2', 'westus', or 'swedencentral'")
print("   2. Re-run from this cell\n")

# Create azd environment and set subscription
subprocess.run(f"azd env new {environment_name} --subscription {subscription_id} --no-prompt", shell=True, capture_output=True)
subprocess.run(f"azd env set AZURE_SUBSCRIPTION_ID {subscription_id}", shell=True, capture_output=True)
subprocess.run(f"azd env set AZURE_LOCATION {location}", shell=True, capture_output=True)
subprocess.run(f"azd env set AZURE_ENV_NAME {environment_name}", shell=True, capture_output=True)

# Set Principal ID
principal_result = subprocess.run("az ad signed-in-user show --query id -o tsv", shell=True, capture_output=True, text=True)

if principal_result.returncode == 0:
    principal_id = principal_result.stdout.strip()
    print(f"👤 Principal ID: {principal_id}")
    subprocess.run(f"azd env set AZURE_PRINCIPAL_ID {principal_id}", shell=True, capture_output=True)
    print("\n✅ All parameters configured successfully!")
else:
    print("❌ Failed to get Principal ID")

## 4. Configure Model

Configure the Azure OpenAI model to deploy. The default is `gpt-4o`, but you can change it to a different model if needed.

> **🎯 Important: Change model in just 1 place here!**  
> Just change the `model_name` and `model_version` variables below, and it will automatically apply to the entire project.  
> You don't need to modify any other files!

### Supported Models

| Model | Version | Features |
|--------|------|------|
| `gpt-4o` | `2024-11-20` | Latest large GA model with structured outputs support (default) |
| `gpt-4o-mini` | `2024-07-18` | Fast and affordable lightweight model, replaces GPT-3.5 Turbo |

**GPT-4o Key Features:**
- **Context Length**: 128,000 tokens input / 16,384 tokens output
- **Multimodal**: Processes text and images
- **Structured Outputs**: Enforces JSON schema
- **JSON Mode** and **Parallel Function Calling**
- Enhanced accuracy and responsiveness
- Excellent performance in non-English languages and vision tasks
- **Enhanced creative writing capabilities**
- Training data: Up to October 2023

**GPT-4o-mini Key Features:**
- **Context Length**: 128,000 tokens input / 16,384 tokens output
- **Fast and affordable**: Designed as GPT-3.5 Turbo replacement
- **Multimodal**: Processes text and images
- **JSON Mode** and **Parallel Function Calling**
- 82% on MMLU benchmark (GPT-3.5 Turbo is 70%)
- Training data: Up to October 2023

> **💡 Note:** 
> - It's recommended to use the latest stable version of each model.
> - Check versions: [Azure OpenAI Models](https://learn.microsoft.com/azure/ai-services/openai/concepts/models)

In [ ]:
# =============================================================================
# 🎯 Model Configuration - Change here and it applies to the entire project!
# =============================================================================

model_name = "gpt-4o"            # 👈 Change model name (e.g., 'gpt-4o-mini')
model_version = "2024-11-20"     # 👈 Change model version (same for all GPT-4o family)
model_capacity = 50              # TPM (Tokens Per Minute) capacity (change if needed)

print("🤖 OpenAI Model Configuration")
print("=" * 50)
print(f"Model Name:     {model_name}")
print(f"Model Version:  {model_version}")
print(f"Capacity (TPM): {model_capacity}")
print("=" * 50)
print("\n✅ This configuration applies to the entire project!")
print("   → Deploy model to Azure OpenAI")
print("   → Auto-generate .env files in Lab 3, 4")
print("   → All Agents use the same model\n")
print("💡 Note:")
print("   • GPT-4o family: gpt-4o, gpt-4o-mini")
print("   • All GPT-4o models use version 2025-08-07")
print("   • Change to corresponding version for other model families\n")

# Set as azd environment variables
subprocess.run(f"azd env set openAiModelName {model_name}", shell=True, capture_output=True)
subprocess.run(f"azd env set openAiModelVersion {model_version}", shell=True, capture_output=True)
subprocess.run(f"azd env set openAiModelCapacity {model_capacity}", shell=True, capture_output=True)

## 5. Deploy Infrastructure

**⚠️ Important Notice:**
- Resources like Azure OpenAI and AI Foundry will be created
- Estimated cost: Approximately $5-10 per hour (varies by usage)
- **This step only creates infrastructure and does not create Container Apps**

### Resources to Deploy

1. **Resource Group** - Group containing all resources
2. **AI Foundry Project** - Agent project workspace
3. **Azure OpenAI** - GPT-4o model (gpt-4o, text-embedding-3-large)
4. **Azure AI Search** - Knowledge base for RAG
5. **Container Apps Environment** - Agent hosting environment (empty environment only)
6. **Container Registry** - Docker image repository
7. **Key Vault** - Secret and key management
8. **Storage Account** - Data storage
9. **Log Analytics Workspace** - Logging and monitoring

### Resources NOT Deployed

- **Container Apps (MCP Server)** - Created with images in Notebook 03

**📝 Note:**
- Container Apps are created in **Notebook 03** when running `azd deploy` with Docker images
- For now, we quickly configure infrastructure only without image pulls (takes about 3-5 minutes)

In [ ]:
print("🚀 Starting Azure infrastructure deployment...")
print(f"Environment: {environment_name}")
print(f"Subscription: {subscription_id}")
print("⏱️  This will take approximately 3-5 minutes (no Container Apps)\n" + "="*50)

result = subprocess.run(
    f"azd provision --no-prompt --environment {environment_name}",
    shell=True,
    text=True,
    env={**os.environ, 'AZURE_SUBSCRIPTION_ID': subscription_id}
)

print("\n" + "="*50)
if result.returncode == 0:
    print("✅ Infrastructure deployment completed successfully!")
    print("\n📋 Created Resources:")
    print("   • AI Foundry Project & OpenAI Models")
    print("   • AI Search Service")
    print("   • Container Apps Environment (empty)")
    print("   • Container Registry")
    print("   • Key Vault, Storage, Monitoring")
    print("\n🚫 NOT Created (will be in Notebook 03):")
    print("   • Container Apps (MCP Server) - will be deployed with images")
    print("\n💡 Next Steps:")
    print("   1. Continue to Section 6: Verify deployed resources")
    print("   2. Notebook 02: Setup AI Search RAG")
    print("   3. Notebook 03: Build images & deploy Container Apps")
else:
    print(f"❌ Deployment failed with return code {result.returncode}")
    print("Check the error messages above for details.")
    print("\n⚠️  If you see 'InsufficientQuota' error:")    
    print("   📌 Alternative regions: eastus2, westus, swedencentral, northeurope")
    print("   1. Go back to Section 3 (Environment Configuration)")    
    print("   2. Change: location = 'eastus2'  # or 'westus', 'swedencentral'")
    print("   3. Re-run Section 3 and Section 5")

## 6. Verify Deployment

Verify the deployed resources and connection information.

In [ ]:
# Get azd environment variables
env_result = subprocess.run(
    "azd env get-values",
    shell=True,
    capture_output=True,
    text=True
)

if env_result.returncode == 0:
    print("=== Deployed Resources Configuration ===\n")
    env_vars = {}
    for line in env_result.stdout.strip().split('\n'):
        if '=' in line:
            key, value = line.split('=', 1)
            # Remove quotes
            value = value.strip('"')
            env_vars[key] = value
            print(f"{key}: {value}")
    
    print("\n" + "="*50)
    print("✅ Environment variables loaded successfully")
else:
    print("❌ Failed to load environment variables")
    env_vars = {}

In [ ]:
# Check resource list in resource group
if 'AZURE_RESOURCE_GROUP' in env_vars:
    rg_name = env_vars['AZURE_RESOURCE_GROUP']
    
    resources_result = subprocess.run(
        f'az resource list --resource-group {rg_name} --query "[].{{Name:name, Type:type, Location:location}}" --output table',
        shell=True,
        capture_output=True,
        text=True
    )
    
    if resources_result.returncode == 0:
        print("=== Deployed Resources ===\n")
        print(resources_result.stdout)
    else:
        print("❌ Failed to list resources")
        print(f"Error: {resources_result.stderr}")
else:
    print("⚠️  Resource group name not found in environment variables")
    print("💡 Please run the previous cell (Cell 17) first to load environment variables.")

## 7. Save Configuration

Save connection information to use in the next notebooks.

In [ ]:
# Extract and save key connection information to config file
# Use minimized environment variables - only essential information for AI Agent execution
config = {
    "resource_group": env_vars.get("AZURE_RESOURCE_GROUP", ""),
    "location": env_vars.get("AZURE_LOCATION", ""),
    "project_connection_string": env_vars.get("AZURE_AI_PROJECT_CONNECTION_STRING", ""),
    "search_endpoint": env_vars.get("AZURE_SEARCH_ENDPOINT", ""),
    "search_service_name": env_vars.get("AZURE_SEARCH_SERVICE_NAME", ""),
    "container_registry_endpoint": env_vars.get("AZURE_CONTAINER_REGISTRY_ENDPOINT", ""),
    "container_apps_environment_id": env_vars.get("AZURE_CONTAINER_APPS_ENVIRONMENT_ID", ""),
    "model_deployment_name": env_vars.get("openAiModelName").strip('"'),
    "model_version": env_vars.get("openAiModelVersion").strip('"'),
    "model_capacity": int(env_vars.get("openAiModelCapacity", "50").strip('"') or "50"),
}

# Save config.json to project root
config_path = Path("config.json")

with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print("=== Configuration Saved (Minimized) ===")
print(f"Location: {config_path.absolute()}")
print("\nEssential Information:")
print(f"  - Resource Group: {config['resource_group']}")
print(f"  - Location: {config['location']}")
print(f"  - AI Project Connection: {'✓ Set' if config['project_connection_string'] else '✗ Missing'}")
print(f"  - Search Endpoint: {config['search_endpoint']}")
print(f"  - Container Registry: {config['container_registry_endpoint']}")
print("\n✅ Minimized configuration saved successfully!")
print("\n💡 Note: Project connection string contains all AI Foundry info (project, OpenAI, etc.)")

## 8. Verify in Azure Portal

You can verify the deployed resources in the Azure Portal.

In [ ]:
# Generate Azure Portal link
resource_group = env_vars.get("AZURE_RESOURCE_GROUP", "")
subscription_id = env_vars.get("AZURE_SUBSCRIPTION_ID", "")

if resource_group and subscription_id:
    portal_url = f"https://portal.azure.com/#@/resource/subscriptions/{subscription_id}/resourceGroups/{resource_group}/overview"
    print("✅ Verify resource group in Azure Portal:")
    print(f"🔗 {portal_url}")
    print("\n🎯 Key Resources:")
    print("  • AI Project: Agent project")
    print("  • Azure OpenAI: GPT-4 model")
    print("  • AI Search: RAG knowledge base")
    print("  • Container Apps Environment: Agent execution environment")
else:
    print("⚠️ Resource group information not found in environment variables.")

## 📍 Next Steps

You have completed deploying Azure resources! Now proceed with the following notebooks in order:

1. **Notebook 02**: Configure RAG knowledge base (`02_setup_ai_search_rag.ipynb`)
2. **Notebook 03**: Deploy Foundry Agent (`03_deploy_foundry_agent.ipynb`)
3. **Notebook 04**: Deploy MAF-based Agent (`04_deploy_foundry_agent_with_maf.ipynb`)
4. **Notebook 05**: MAF workflow patterns hands-on (`05_maf_workflow_patterns.ipynb`)
5. **Notebook 06**: MAF Dev UI hands-on (`06_maf_dev_ui.ipynb`)
6. **Notebook 07**: Agent evaluation (`07_evaluate_agents.ipynb`)